# mpl_map
---
Plots a simple, static ashfall model for a given time step that can be projected into an arbitrary coordinate system. Good for quick model evaluations. Strangely, setting tick labels and gridlines is limited to only two projections:

`TypeError: Cannot label gridlines on a _EPSGProjection plot.  Only PlateCarree and Mercator plots are currently supported.`

This limitation will apparently be [addressed sometime in the future](http://scitools.org.uk/cartopy/docs/v0.13/matplotlib/gridliner.html#cartopy-map-gridlines-and-tick-labels), but meanwhile, GeoViews with the Matplotlib backend handles tick labels for arbitrary projections just fine.

Overall, the combination of Matplotlib and cartopy can compete with GMT from an aesthetic standpoint, and it is certainly much more reasonable than GMT in terms of ease-of-use and code clarity (GMT may address these issues with [GMT/Python](http://www.gmtpython.xyz/)).

## *Pros*
* Minimal code required to produce a very nice product
* Plays well with cartopy &mdash; that means features, projections etc.
* Integrates quite nicely with xarray plotting (for example, this notebook uses [`xarray.plot.pcolormesh`](http://xarray.pydata.org/en/stable/generated/xarray.plot.pcolormesh.html#xarray-plot-pcolormesh))

## *Cons*
* Not interactive
* Poor support for tick labels and gridlines

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import colorcet as cc
import cartopy.feature as cf 
import cartopy.crs as ccrs

import sys
sys.path.insert(0, '../')
from vis_tools import read_hysplit_netcdf

# ignore warning that arises from my use of NaN's instead of 0's
import warnings
warnings.filterwarnings('ignore', message='invalid value encountered in less_equal')

###############################################
# SPECIFY: file name and path for HYSPLIT model
FILENAME = '../18042918_taupo_15.0_0.01.nc'

# SPECIFY: pick one time step to plot
TIME_STEP = 8

# SPECIFY: plotting params
ASH_MIN = 10**-1  # min ash colorbar cutoff
ASH_MAX = 10**2  # max ash colorbar cutoff
###############################################

model = read_hysplit_netcdf(FILENAME, ASH_MIN)

NZTM_PROJ = ccrs.epsg(2193)  # define projection using NZTM EPSG code (need internet access)

# create figure and set font size
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=NZTM_PROJ)

# PLOT background geography
ax.coastlines(resolution='50m', zorder=3)
ax.add_feature(cf.NaturalEarthFeature(category='physical', scale='50m', name='lakes',
                                      edgecolor='k', facecolor='none', zorder=4))

# PLOT ashfall (using log scale)
ash = model.isel(time=TIME_STEP)['total_deposition']
ash.plot.pcolormesh(transform=ccrs.PlateCarree(), cmap=cc.m_fire_r, cbar_kwargs=dict(label='ash thickness (mm)'),
                    norm=LogNorm(), vmin=ASH_MIN, vmax=ASH_MAX, zorder=1)
contour_levels = np.arange(np.log10(ASH_MIN)+1, np.log10(ASH_MAX)+1)  # log-spaced contours (skip ASH_MIN contour)
ash.plot.contour(ax=ax, transform=ccrs.PlateCarree(), levels=contour_levels,
                 colors='k', alpha=0.5, linewidths=0.75, zorder=2)

# PLOT source location
ax.plot(*model.attrs['volcano_location'][::-1], transform=ccrs.Geodetic(),
        marker='^', ms=10, mew=0.75, c='c', mec='k', zorder=5)

ts = np.mean(np.ediff1d(model['time']).astype('timedelta64[h]')).astype(int)  # hours

plt.title(FILENAME.split('/')[-1] + '\n' + str(TIME_STEP*ts) + ' hours after eruption onset');